In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np 
from pathlib import Path
from bs4 import BeautifulSoup
import re, time
import requests
tqdm.pandas()
import tldextract
from selenium import webdriver
import undetected_chromedriver as uc

path = 'data'

root_folder = Path(path)
root_folder.mkdir(parents=True, exist_ok=True)

In [7]:
files = list(root_folder.rglob("*"))  
files = [f for f in files if f.is_file() and f.suffix == ".html"]

df_searchterms = pd.read_csv(path+'/google_scholar_search_base.csv',sep =';')
lst_name_files = [file.name for file in files]
df_searchterms = df_searchterms[df_searchterms['file'].isin(lst_name_files)].copy()
df_searchterms['path_file'] = path +'/raw_data/'+ df_searchterms['folder']+'/'+df_searchterms['file']

In [8]:


def process_file(path_file):
    f = open(path_file,"r", encoding="utf-8")
    soup = BeautifulSoup(f.read(), "html.parser")
    f.close()

    lst_divs_resultado_busca = soup.find_all("div", attrs={"data-rp": True})

    lst_records = []

    for bloco_resultado in lst_divs_resultado_busca:

        main_div = bloco_resultado.find('div',{'class':'gs_ri'})
        record = {}

        record['href'] = None
        href_div = main_div.find('a', href = True)
        if href_div:
            record['href'] = href_div['href']

        record['title'] = None
        title_div =  main_div.find('h3')
        if title_div:
            record['title'] = title_div.text

        record['author_div'] = None
        author_div = main_div.find('div',{'class':'gs_a'})
        if author_div:
            record['author_div'] = author_div.text

        record['description'] = None
        description_div = main_div.find('div',{'class':'gs_rs'})
        if description_div:
            record['description'] = description_div.text


        record['citations'] = 0
        citations_div = main_div.find('div',{'class':'gs_fl gs_flb'})
        if citations_div:
            match = re.search(r"citado por (\d+)", citations_div.text, re.IGNORECASE)
            if match:
                record['citations'] = int(match.group(1))

        secondary_div = bloco_resultado.find('div',{'class':'gs_ggs gs_fl'})

        record['direct_label'] = None 
        record['direct_href'] = None 
        if secondary_div:
            record['direct_label'] = secondary_div.text
            direct_href_div = secondary_div.find('a',href = True)

            if direct_href_div :
                record['direct_href'] = direct_href_div['href']

        lst_records.append(record)

    return lst_records #pd.DataFrame(lst_records)

df_searchterms['df'] = df_searchterms['path_file'].progress_map(lambda x : process_file(x))
df_searchterms = df_searchterms.explode('df')
for key in ['href', 'title', 'author_div', 'description', 'citations','direct_label','direct_href']:
    df_searchterms[key] = df_searchterms['df'].map(lambda x : x[key])
df_searchterms.drop(columns = 'df', inplace=True)

  0%|          | 0/810 [00:00<?, ?it/s]

In [9]:
df_searchterms['PDF'] = df_searchterms['direct_label'].str.contains('PDF', regex=False).astype('bool')
df_searchterms['target_url'] = df_searchterms['direct_href'].fillna(df_searchterms['href'])


ix = df_searchterms['target_url'].str.startswith('https://arxiv.org')
df_searchterms.loc[ix,'PDF'] = True 
df_searchterms['target_url'] = df_searchterms['target_url'].str.replace('abs','pdf')

ix = df_searchterms['target_url'].str.endswith('.pdf')
df_searchterms.loc[ix,'PDF'] = True 

ix = df_searchterms['target_url'].str.startswith('https://dl.acm.org/doi/pdf/')
df_searchterms.loc[ix,'target_url'] = df_searchterms.loc[ix,'target_url'].str.replace("pdf/",'', regex = False)
df_searchterms.loc[ix,'PDF'] = False 

ix = df_searchterms['target_url'].str.contains('sciencedirect')
df_searchterms.loc[ix,'PDF'] = False 

ix = df_searchterms['target_url'].str.contains('/science/article/am/', regex =False)
df_searchterms.loc[ix,'target_url'] = df_searchterms.loc[ix,'target_url']\
        .str.replace('/science/article/am/','/science/article/abs/', regex =False) 

df_searchterms['article_file'] = (
    df_searchterms['target_url'].str.slice(0,100)
    .str.replace(r'[^a-zA-Z0-9_]', '_', regex=True)  # substitui caracteres especiais por _
    .str.replace(r'_+', '_', regex=True)             # múltiplos _ por 1 _
    .str.strip('_')                         # remove _ no começo/fim
)

ix = df_searchterms['PDF'] 
df_searchterms.loc[ix,'article_file']+='.pdf'
df_searchterms.loc[~ix,'article_file']+='.html'




In [16]:
ix = df_searchterms['target_url'].notnull()
df_searchterms.loc[ix,'domain'] = df_searchterms.loc[ix,'target_url'].map(lambda x:tldextract.extract(x).domain ) 

df_searchterms['domain_count'] = df_searchterms.groupby('domain')['domain'].transform('count')

df_searchterms.sort_values(by = ['domain_count','domain','folder'],\
                            ascending=[False,False,False], ignore_index=True, inplace=True)

In [17]:
df_searchterms.to_csv(path+'/google_scholar_search_base_article_detail.csv', sep = ';', index = False)

In [18]:

df_searchterms['full_path_folder'] =  str(root_folder) + '/articles/' + df_searchterms['folder'] 
df_searchterms['full_path'] = df_searchterms['full_path_folder'] + '/'+ df_searchterms['article_file']
files = list((root_folder / 'articles').rglob("*"))  
files = [str(f) for f in files if f.is_file() and f.suffix in (".html",'.pdf')]
s = df_searchterms['full_path'].map(lambda x : str(Path(x)))
df_searchterms = df_searchterms[~s.isin(files)]

In [19]:
records = df_searchterms.to_dict(orient = 'records')

In [20]:
# Inicializa o navegador
#driver = webdriver.Chrome()
driver = uc.Chrome(version_main=132)

In [10]:
df_searchterms['target_url'].values[0]

'https://arxiv.org/pdf/2306.05949'

In [21]:
url = df_searchterms['target_url'].values[0]
driver.get(url)

In [13]:
driver.get(url)

In [23]:
block_domains = set()

for record in tqdm(records):


    full_path_folder =  Path(record['full_path_folder'])
    full_path_folder.mkdir(parents=True, exist_ok=True)
    full_path = Path(record['full_path'])

    if full_path.exists() and full_path.is_file():  
        continue
    
    url = record['target_url']
    if not isinstance(url, str):
        continue 

    if record['domain'] in block_domains:
        continue

    error = False

    if record['PDF']:
        try:
            response = requests.get(url,  timeout=20)

            if response.status_code == 200:
                with open(full_path, "wb") as f:
                    f.write(response.content)
            else:
                error = True
                    
        except Exception as e:
            print('erro em ',url)
            print(e)
            error = True

    else:
        try:
            driver.get(url)
            time.sleep(4)
            alpha, beta = 2,5
            val = np.random.beta(alpha, beta)
            wait_time = 1 + val * (3 - 1)
            time.sleep(wait_time) 

            if 'Confirme que você é humano' in driver.page_source or\
                'Verificando se você é humano' in driver.page_source or\
                'There was a problem providing the content you requested' in driver.page_source or\
                'but we are experiencing unusual' in driver.page_source or\
                'we require CAPTCHA on specific webpages' in driver.page_source:
                block_domains.add(record['domain'])
                print('domain block ', record['domain'])
                continue


            with open(full_path, "w", encoding="utf-8") as f:
                f.write(driver.page_source)
        except Exception as e:
            print('erro em ',url)
            print(e)
            error = True

    #if error:
    #    with open(full_path, "w") as f:
    #        f.write('')

    

    

        


  0%|          | 0/2863 [00:00<?, ?it/s]

erro em  javascript:void(0)
Message: unknown error: unsupported protocol
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x00C374A3+25091]
	(No symbol) [0x00BBDC04]
	(No symbol) [0x00A9B373]
	(No symbol) [0x00A9DCF8]
	(No symbol) [0x00B1BEC7]
	(No symbol) [0x00B01F0C]
	(No symbol) [0x00B1B49E]
	(No symbol) [0x00B01CA6]
	(No symbol) [0x00AD31D5]
	(No symbol) [0x00AD435D]
	GetHandleVerifier [0x00F307C3+3142947]
	GetHandleVerifier [0x00F41A2B+3213195]
	GetHandleVerifier [0x00F3C412+3191154]
	GetHandleVerifier [0x00CD8720+685184]
	(No symbol) [0x00BC6E1D]
	(No symbol) [0x00BC3E18]
	(No symbol) [0x00BC3FB6]
	(No symbol) [0x00BB66F0]
	BaseThreadInitThunk [0x76695D49+25]
	RtlInitializeExceptionChain [0x7773D03B+107]
	RtlGetAppContainerNamedObjectPath [0x7773CFC1+561]

erro em  javascript:void(0)
Message: unknown error: unsupported protocol
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x00C374A3+25091]
	(No symbol) [0x00BBDC04]
	(No symbol) [0